In [1]:
for ii in 1:2
    addprocs(["heathred"])
end

for ii in 1:1
    addprocs(["amon"], dir="")
    addprocs(["zeus"], dir="")
    addprocs(["jove"], dir="")
    addprocs(["ares"], dir="")
end

for ii in 1:10
    #addprocs(["uggp"], dir="")
end

addprocs(6)
workers()

expr: syntax error
expr: syntax error
expr: syntax error
expr: syntax error


12-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13

In [2]:
using Iterators
using Pipe
function pz(x :: AbstractArray)
    println(typeof(x), ": ", size(x))
end
macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [3]:
@everywhere using RecursiveAutoencoders
@everywhere  using UnfoldingRAE

include("ClusterSoup.jl")

get_chunks (generic function with 1 method)

In [4]:
using WordEmbeddings
LL,word_indexes, indexed_words =  load_embeddings("embeddings-scaled.EMBEDDING_SIZE=50.txt");
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(50,268810)
Dict{String,Int64}
Array{String,1}


In [5]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@assert length(@pipe training_trees |> filter(x->typeof(x)===UTF8String,_))==0
@pz training_trees

training_trees		Array{Any,1}	(3913,)


In [6]:
chunk_data(:training_trees, training_trees)

2: Array{Any,1}: 327
3: Array{Any,1}: 326
4: Array{Any,1}: 326
5: Array{Any,1}: 326
6: Array{Any,1}: 326
7: Array{Any,1}: 326
8: Array{Any,1}: 326
9: Array{Any,1}: 326
10: Array{Any,1}: 326
11: Array{Any,1}: 326
12: Array{Any,1}: 326
13: Array{Any,1}: 326


In [7]:
@everywhere function unpack!(rae::RAE, θ::Vector)
    W_e_len = length(rae.W_e)
    b_e_len = length(rae.b_e)
    W_d_len = length(rae.W_d)
    b_d_len = length(rae.b_d)
    W_e_shape = size(rae.W_e)
    W_d_shape = size(rae.W_d)
    
    rae.W_e = reshape(θ[1: W_e_len],W_e_shape)
    rae.b_e = θ[W_e_len+1: W_e_len+b_e_len]
    rae.W_d = reshape(θ[W_e_len+b_e_len+1: W_e_len+b_e_len+W_d_len],W_d_shape)
    rae.b_d = θ[W_e_len+b_e_len+W_d_len+1: end]
    
    rae
end

@everywhere function pack(rae::RAE)
    pack(rae.W_e,rae.b_e, rae.W_d,rae.b_d)
end

@everywhere function pack(∇W_e::Matrix{Float64}, ∇b_e::Vector{Float64}, ∇W_d::Matrix{Float64}, ∇b_d::Vector{Float64})
    [∇W_e[:], ∇b_e, ∇W_d[:], ∇b_d] 
end


In [29]:

function loss!(θ::Vector)  
    #warn("loss! not defined")
    grad = similar(θ)
    #loss_and_loss_grad!(θ::Vector)
    cached_loss_and_loss_grad!(θ, grad)
end

function loss_grad!(θ::Vector, storage::Vector) 
    #warn("loss_grad not defined")
    cached_loss_and_loss_grad!(θ, grad)
end


#rae_outer = RAE(LL,word_indexes,indexed_words);
#set_global(:rae_outer,rae_outer)
function loss_and_loss_grad!(θ::Vector, grad::Vector)   
    grad[:] = 0
    for pid in workers()
        remotecall(pid, θv->unpack!(rae_outer, θv),θ) 
    end
    
    
    function loss_and_loss_grad_single(tree::(Any,Any))
        Δs, err = UnfoldingRAE.loss_and_loss_grad(rae_outer, tree)
        [pack(Δs...), err]
    end
    
    ret = prechunked_mapreduce(:training_trees, loss_and_loss_grad_single, (+)) 
    grad[:] = ret[1:end-1]
    err=ret[end]
    
    grad[:]/=length(training_trees)
    err/=length(training_trees)
    err
end

loss_and_loss_grad! (generic function with 1 method)

In [37]:

loss_and_loss_grad_cache = Dict{Vector{Float64},(Float64, Vector{Float64})}()
loss_and_loss_grad_cache_hits = 0
loss_and_loss_grad_cache_misses = 0
function cached_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global loss_and_loss_grad_cache
    global loss_and_loss_grad_cache_hits
    global loss_and_loss_grad_cache_misses
    if haskey(loss_and_loss_grad_cache,θ)
        loss_and_loss_grad_cache_hits+=1
        err, grad[:] = loss_and_loss_grad_cache[θ]
        err
    else
        loss_and_loss_grad_cache_misses+=1
        err = loss_and_loss_grad!(θ, grad)
        loss_and_loss_grad_cache[θ] = (err, grad)
        err
    end
end

cached_loss_and_loss_grad! (generic function with 1 method)

In [25]:
using NLopt

f_call_count = 0
function tracking_loss_and_loss_grad!(θ::Vector, grad::Vector)
    global f_call_count
    f_call_count+=1
    f_val = cached_loss_and_loss_grad!(θ, grad)   
    println(f_call_count, '\t',f_val,'\t',norm(grad))
    f_val
end
#:LD_MMA, :LD_CCSAQ, :LD_LBFGS, :LD_SLSQP, :LD_VAR2, :LD_VAR1, :LD_TNEWTON_RESTART
opt = Opt(:LD_MMA, length(pack(rae_outer)))

#ftol_abs!(opt,1e-9)
maxtime!(opt, 60*2)
min_objective!(opt, tracking_loss_and_loss_grad!)
(optf,optx,ret) = optimize!(opt,pack(rae_outer))


1	636.1843247007678	6.3573527328668884e10
2	675.0712809776059	1.6934258781047438e14
3	674.9869275278155	1.018445077529498e15
4	675.1855474529774	6.624529721239418e16
5	675.0284989280419	1.3713330359652124e18
6	674.8836018575004	5.217157715228289e16
7	673.4248730746976	7.669958250468692e15
8	670.0659293269231	2.0737192259397572e16
9	666.3042419706875	9.400423613012953e11
10	675.0712809776059	9.400423613012953e11
11	675.1855474529774	9.400423613012953e11
12	670.0659293269231	9.400423613012953e11
13	675.0712809776059	9.400423613012953e11
14	670.0659293269231	9.400423613012953e11
15	650.5794761043388	1.0454138368271562e11
16	644.8296109554103	4.2746263797998245e10


(636.1843247007678,[0.00678168,-0.0054377,-0.0174587,-0.0112756,-0.00328987,-0.0127259,0.0137251,0.0123097,-0.012145,-6.45478e-5  …  -0.00545221,0.00192604,-0.0181612,-0.00416915,0.00966524,0.000922916,-0.0044354,-0.00691362,0.0190044,-0.00950634],:MAXTIME_REACHED)

In [ ]:
#MMA
#167	623.6986598947503	3.472825423124687e12

In [47]:
using Optim #https://github.com/JuliaOpt/Optim.jl
f=DifferentiableFunction(loss!,loss_grad!,cached_loss_and_loss_grad!)

res = optimize(f, res.minimum, method=:l_bfgs, show_trace = true, store_trace = true, iterations = 30);
@printval res.f_calls 
@printval res.g_calls 
@printval res.iterations
@printval res.f_minimum
@printval res.gr_converged
@printval res.x_converged 

@printval res.trace
@printval loss_and_loss_grad_cache_hits
@printval loss_and_loss_grad_cache_misses

Iter     Function value   Gradient norm 
     0     6.351856e+02     5.653994e+08
     1     6.343637e+02     2.445336e+09
     2     6.343586e+02     1.853079e+09
     3     6.343585e+02     3.475596e+09
     4     6.343585e+02     3.166523e+09
     5     6.343584e+02     8.634659e+09
     6     6.343583e+02     1.111431e+10
     7     6.343578e+02     2.852625e+10
     8     6.343571e+02     3.742282e+10
     9     6.343568e+02     6.047751e+09
    10     6.343574e+02     3.516779e+10
    11     6.343558e+02     2.899287e+10
    12     6.343558e+02     3.183960e+10
    13     6.343542e+02     2.462507e+10
    14     6.343513e+02     5.889620e+09
    15     6.343498e+02     2.936181e+09
    16     6.343495e+02     3.051671e+09
    17     6.343495e+02     3.009379e+09
    18     6.343494e+02     3.808102e+09
    19     6.343494e+02     2.660618e+09
    20     6.343495e+02     4.103978e+09
    21     6.343495e+02     4.120164e+09


Write failed: Broken pipe
Write failed: Broken pipe
Write failed: Broken pipe
Write failed: Broken pipe


    22     6.343495e+02     3.689587e+09
    23     6.343495e+02     3.065983e+09
    24     6.343445e+02     3.793618e+09
    25     6.343439e+02     1.957278e+10
    26     6.343443e+02     3.938279e+09
    27     6.343443e+02     5.481373e+09
    28     6.343443e+02     6.615160e+09
    29     6.343439e+02     1.179713e+10
    30     6.343414e+02     1.492140e+11
res.f_calls = 366
res.g_calls = 366
res.iterations = 30
res.f_minimum = 634.3414419524082
res.gr_converged = false
res.x_converged = false
res.trace = Iter     Function value   Gradient norm 
------   --------------   --------------
     0     6.351856e+02     5.653994e+08
     1     6.343637e+02     2.445336e+09
     2     6.343586e+02     1.853079e+09
     3     6.343585e+02     3.475596e+09
     4     6.343585e+02     3.166523e+09
     5     6.343584e+02     8.634659e+09
     6     6.343583e+02     1.111431e+10
     7     6.343578e+02     2.852625e+10
     8     6.343571e+02     3.742282e+10
     9     6.343568e+02     6

In [46]:
res

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [0.0067816778734018325,-0.005437701228900908,-0.017458747230653392,-0.011275588891105968,-0.003289866038265289,-0.012725850827617229,0.013725112899514154,0.012309720984219232,-0.012145011236904948,-6.454781792256092e-5,0.026487730944640915,-0.01715740756986214,0.0050860099860759725,0.006012645462656855,-0.009188278337591784,0.00238639676336448,0.0024845169179436907,0.000658613210695495,-0.004974335303875942,-0.0016803419439271072,-0.006574048416179822,0.024823231188016447,0.005041673212411766,0.008706938323507019,-0.005693782481273362,0.01455499581344127,0.01896593043454144,-0.001923081429400532,-0.010769598573247023,0.002269308128235589,-0.006851905075449971,0.011796303758407503,-0.0022937784406319084,-0.015413375510162712,0.009670850761218678,0.001645009465295914,-0.013979252498346932,-0.004319499516777195,-0.002939182543234947,-0.006654789492225426,-0.01680798833056237,0.009295078379467237,-0.00596457719440541

In [ ]:
Profile.clear()

In [ ]:
θ=pack(rae_outer)
@time loss_and_loss_grad!(θ, zeros(size(θ)))